# NLP Project 2022

## Get data

In [ ]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/VaxMisinfoData.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
#  !pip install twarc
#  !twarc configure

Didn't work because of authentication

In [ ]:
'''
from twarc import Twarc
from urllib.request import urlretrieve

tweet_ids_url = 'https://raw.githubusercontent.com/SakibShahriar95/ANTiVax/main/Labeled/ids.txt'
tweet_ids = urlretrieve(tweet_ids_url, './ids.txt') # it's a file like object and works just like a file

df = pd.read_csv('./ids.txt')
df.head()

consumer_key="2lNba8Nvu9f0i5YgfmQhfh3mP"
consumer_secret="TGVv9QJhtkjSqm7QyMeLfpZm3IU3xjrrMabyae42VxbkaT5t0g"
access_token="1600730532-ZCeeHr9h033QRlC95BLZrpz85TRFN4pZ0w7vGhY"
access_token_secret="eoiufziPLBr0eYgR12gAoAEiTQjlnTkZ1BHBLXoDuQbtC"
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

df = pd.DataFrame()
for tweet in t.hydrate(open('./ids.txt')):
  row = { 'id': tweet['id'], 'text': tweet['text'] }
  df.append(row, ignore_index = True)
'''


We hydrated offline and saved it to Github to retrieve it from there

In [ ]:
url = 'https://raw.githubusercontent.com/krautgortna/NLP_Project_SS22/main/data/hydrated_tweets.csv'
all_data = pd.read_csv(url)
all_data.text

In [ ]:
Total = all_data.isnull().sum().sort_values(ascending=False)
percent = (all_data.isnull().sum() / all_data.isnull().count()).sort_values(ascending=False)
missing_stats = pd.concat([Total, percent], axis=1, keys=['Total', 'Percent'])

all_data_drop_80_missing = all_data.drop((missing_stats[missing_stats['Percent'] > .8]).index, axis=1)
missing_stats.head(25)

In [ ]:
df_merged = df.merge(all_data[['id', 'text', 'hashtags']], on='id')
df_merged


In [ ]:
list(df_merged['hashtags'].str.split(" ").explode('hashtags').unique())

# EDA

## Preprocessing

In [ ]:
import re

def remove_targets(review_text):
    return re.sub('(@[A-Za-z0-9])\w+', '', review_text)

def remove_hashtags(review_text):
    return review_text.replace("#", " ")

def remove_numbers(review_text):
    return re.sub('[0-9]', '', review_text)

def remove_punctuation(review_text):
    return re.sub(r'[/\[\]\\\{\}\(\)\|\;\'\:\"\,\!\?\.]', '', review_text)

def remove_newlines(review_text):
    return review_text.replace("\n", " ")

df_cleaned = df_merged.copy(deep=True)

df_cleaned['text'] = df_cleaned['text'].apply(remove_targets)
df_cleaned['text'] = df_cleaned['text'].apply(remove_hashtags)
df_cleaned['text'] = df_cleaned['text'].apply(remove_numbers)

df_cleaned['text'] = df_cleaned['text'].apply(remove_punctuation)
df_cleaned['text'] = df_cleaned['text'].apply(remove_newlines)

df_cleaned['text'] = df_cleaned['text'].str.encode('ascii', 'ignore').str.decode('ascii')
df_cleaned['text'] = df_cleaned['text'].str.lower()

df_cleaned

## Train model

## Tuning

## Debugging